In [114]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [49]:
tmdb_movies = pd.read_csv('data/tmdb/tmdb_5000_movies.csv')
tmdb_credits = pd.read_csv('data/tmdb/tmdb_5000_credits.csv')

In [50]:
tmdb_movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [51]:
tmdb_credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [52]:
print(tmdb_movies.shape)
print(tmdb_credits.shape)

(4803, 20)
(4803, 4)


# FIND SIMILARTY FOR OVERVIEWS

In [53]:
tmdb_credits.columns = ['id','title','cast','crew']
tmdb_credits.head(3)

,id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [54]:
tmdb_all_data = tmdb_movies.merge(tmdb_credits, on='id')
tmdb_all_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [122]:
tmdb_all_data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title_x', 'vote_average',
       'vote_count', 'title_y', 'cast', 'crew'],
      dtype='object')

In [55]:
tmdb_all_data['overview'] = tmdb_all_data['overview'].fillna('')

In [56]:
tfidf = TfidfVectorizer(stop_words='english')

In [57]:
transformed_overviews = tfidf.fit_transform(tmdb_all_data['overview'])
transformed_overviews.shape

(4803, 20978)

### We use linear kernel to find similarity between overviews because we need dot product here, and it gives us the cosine similarity score.

In [63]:
cosine_similarities = linear_kernel(transformed_overviews, transformed_overviews)
cosine_similarities.shape

(4803, 4803)

In [174]:
def get_movie_recommendation_with_similarities(title, similarity, top=10):
    idx = tmdb_all_data[tmdb_all_data['title_x'] == title].index[0]
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top+1]
    movie_indices = [i[0] for i in sim_scores]
    return tmdb_all_data['title_x'].iloc[movie_indices]
    

### Let's get a random movie from the dataset and find the top 10 similar movies.

In [113]:
random_movie = tmdb_all_data.sample(1)['title_x'].values[0]
print('Movie: ', random_movie)
get_movie_recommendation_with_similarities(random_movie, cosine_similarities, top=10)

Movie:  Veronika Decides to Die


2118                  Premonition
4783                     Backmask
1015                        K-PAX
4735            The Night Visitor
3423              Dressed to Kill
1654                      Godsend
4629    Some Guy Who Kills People
1387                    Malcolm X
2477              Jennifer's Body
845                      Instinct
Name: title_x, dtype: object

### We will use the other features to find the similarity.

#### We will use count vectorizer to convert other features to one vector. Let's see how it works.

In [119]:
vectorizer = CountVectorizer()
sampleData = ['This is the first document.','This document is the second document.','And this is the third one.','Is this the first document?',]
X = vectorizer.fit_transform(sampleData)
print(vectorizer.get_feature_names())
print(X.toarray())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


In [120]:
vectorizer = CountVectorizer()

### Now we know how to convert the features to vectors. Let's concat all features to one data and convert it to vector.

### Get cast, crew, keywords and genre names from the dataset.

In [159]:
new_data = tmdb_all_data.copy()
new_data['cast'] = new_data['cast'].apply(lambda x: [i['name'] for i in json.loads(x)] if isinstance(x, str) else [])
new_data['crew'] = new_data['crew'].apply(lambda x: [i['name'] for i in json.loads(x)] if isinstance(x, str) else [])
new_data['genres'] = new_data['genres'].apply(lambda x: [i['name'] for i in json.loads(x)] if isinstance(x, str) else [])
new_data['keywords'] = new_data['keywords'].apply(lambda x: [i['name'] for i in json.loads(x)] if isinstance(x, str) else [])

new_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[Stephen E. Rivkin, Rick Carter, Christopher B..."
1,300000000,"[Adventure, Fantasy, Action]",http://disney.go.com/disneypictures/pirates/,285,"[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[Dariusz Wolski, Gore Verbinski, Jerry Bruckhe..."
2,245000000,"[Action, Adventure, Crime]",http://www.sonypictures.com/movies/spectre/,206647,"[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[Thomas Newman, Sam Mendes, Anna Pinnock, John..."


### Convert related features to strings for count vectorizer.

In [160]:
new_data['genres'] = new_data['genres'].apply(lambda x: ' '.join(x))
new_data['cast'] = new_data['cast'].apply(lambda x: ' '.join(x))
new_data['crew'] = new_data['crew'].apply(lambda x: ' '.join(x))
new_data['keywords'] = new_data['keywords'].apply(lambda x: ' '.join(x))
new_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,Stephen E. Rivkin Rick Carter Christopher Boye...
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,Dariusz Wolski Gore Verbinski Jerry Bruckheime...
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,Daniel Craig Christoph Waltz Léa Seydoux Ralph...,Thomas Newman Sam Mendes Anna Pinnock John Log...


### Add all features to one data.

In [167]:
new_data['concated_data'] = new_data['genres'] + ' ' + new_data['cast'] + ' ' + new_data['crew'] + ' ' + new_data['keywords']
new_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew,newdata,concated_data
0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,Stephen E. Rivkin Rick Carter Christopher Boye...,Action Adventure Fantasy Science Fiction Sam W...,Action Adventure Fantasy Science Fiction Sam W...
1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,Dariusz Wolski Gore Verbinski Jerry Bruckheime...,Adventure Fantasy Action Johnny Depp Orlando B...,Adventure Fantasy Action Johnny Depp Orlando B...
2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6 bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,Daniel Craig Christoph Waltz Léa Seydoux Ralph...,Thomas Newman Sam Mendes Anna Pinnock John Log...,Action Adventure Crime Daniel Craig Christoph ...,Action Adventure Crime Daniel Craig Christoph ...


In [168]:
count_matrix = vectorizer.fit_transform(new_data['concated_data'])
count_matrix.shape

(4803, 62236)

In [169]:
cosine_similarity_for_concated_data = cosine_similarity(count_matrix, count_matrix)

### Let's see how it looks like with some random movies.

In [172]:
random_movie = tmdb_all_data.sample(1)['title_x'].values[0]
print('Movie: ', random_movie)
get_movie_recommendation_with_similarities(random_movie, cosine_similarity_for_concated_data, top=10)

Movie:  Pirates of the Caribbean: On Stranger Tides


1                Pirates of the Caribbean: At World's End
199     Pirates of the Caribbean: The Curse of the Bla...
12             Pirates of the Caribbean: Dead Man's Chest
1990                              The Empire Strikes Back
322                                     The Fifth Element
197              Harry Potter and the Philosopher's Stone
19              The Hobbit: The Battle of the Five Armies
50                    Prince of Persia: The Sands of Time
169                    Captain America: The First Avenger
149                                            Armageddon
Name: title_x, dtype: object

In [177]:
random_movie = tmdb_all_data.sample(1)['title_x'].values[0]
print('Movie: ', random_movie)
get_movie_recommendation_with_similarities(random_movie, cosine_similarity_for_concated_data, top=10)

Movie:  Friday the 13th Part III


4282              Friday the 13th Part 2
3364                             Warlock
3314                            Shopgirl
740                  Alien: Resurrection
853            Elizabeth: The Golden Age
3462                      Daddy Day Camp
1133                          15 Minutes
829                            Bowfinger
2405     I Know What You Did Last Summer
4053    Friday the 13th: A New Beginning
Name: title_x, dtype: object

### Let's see more random movies.

In [181]:
for i in range(10):
    random_movie = tmdb_all_data.sample(1)['title_x'].values[0]
    print('Movie: ', random_movie)
    print(get_movie_recommendation_with_similarities(random_movie, cosine_similarity_for_concated_data, top=10))
    print('-------------------------')

Movie:  Space Battleship Yamato
1709     Space Pirate Captain Harlock
4401              The Helix... Loaded
97                      Shin Godzilla
4344                    Godzilla 2000
2292    Thomas and the Magic Railroad
3542                             R100
4728     The Lost Skeleton of Cadavra
4187           Independence Daysaster
1715                          Timecop
2577                 Tuck Everlasting
Name: title_x, dtype: object
-------------------------
Movie:  The Bounty Hunter
551                           Fool's Gold
790                       American Sniper
1526                   Going the Distance
1299                   Sweet Home Alabama
609                           Escape Plan
815                                 Hitch
932                        V for Vendetta
38               The Amazing Spider-Man 2
4235    I Want Someone to Eat Cheese With
1826                          Cloverfield
Name: title_x, dtype: object
-------------------------
Movie:  The Full Monty
276      